<img src="6 - Estimation.assets/image-20220818151351911.png" alt="image-20220818151351911">

# 0. Notation

* CATEs : Conditional average treatment effects
  * $\tau(x) := \mathbb E [Y(1) - Y(0) \mid X=x]$
* Positivity & NUC
  * $\tau := \mathbb E_W[\mathbb E_Y [Y\mid T=1, W] - \mathbb E_Y [Y\mid T=0, W]]$
    * $W$ is a sufficient adjustment set
  * $\tau(x) := \mathbb E_W[\mathbb E_Y [Y\mid T=1, X=x, W] - \mathbb E_Y [Y\mid T=0, X=x, W]]$
    * $W \cup X$ is a sufficient adjustment set

# 1. Conditional Outcome Modeling

* $\tau := \mu(1, W) - \mu(0, W)$
* Conditional Outcome Model (COM)
  * $\hat \mu$
    * linear model
    * neural network
    * etc.
* COM estimator for the
  * ATE
    * $\hat \tau := \frac{1}{n} \sum_i (\hat \mu (1, w_i) - \hat \mu (0, w_i) )$
  * CATE
    * $\hat \tau(x) := \frac{1}{n_x} \sum_{i: x_i= x} (\hat \mu (1, w_i, x) - \hat \mu (0, w_i, x) )$
* COM estimation은 많은 이름으로 불림
  * G-computation estimator
  * Parametric G-formula
  * S-learner
  * etc.

## 1.1 Problem with COM estimation in high dims

<img src="6 - Estimation.assets/image-20220821161215239.png" alt="image-20220821161215239">

* $T$가 1차원이라 $T$없이 $W$만으로 예측하는 머신이 나올 수 있음
* 그 경우, $\hat \tau \approx 0$이 나오고 estimator가 biased임을 암시

## 1.2 Grouped COM (GCOM) estimation

<img src="6 - Estimation.assets/image-20220821162102383.png" alt="image-20220821162102383">

* GCOM estimator for the
  * ATE
    * $\hat \tau := \frac{1}{n} \sum_i (\hat \mu_1 (w_i) - \hat \mu_0 (w_i) )$
  * CATE
    * $\hat \tau(x) := \frac{1}{n_x} \sum_{i: x_i= x} (\hat \mu_1 (w_i, x) - \hat \mu_0 (w_i, x) )$

* 두 개의 모델을 따로 학습함
* 데이터 효율이 떨어지고 높은 분산을 야기함

## 1.3. Increasing Data Efficiency

### 1.3.1 TARNet

<img src="6 - Estimation.assets/image-20220821165410787.png" alt="image-20220821165410787">

* 모든 데이터로 Representation Learning은 하지만,
* Predictor 학습은 따로한 후에
* COM estimator로 활용
  * ATE
    * $\hat \tau = \frac{1}{n} \sum_i (\hat \mu (1, w_i) - \hat \mu (0, w_i) )$
  * CATE
    * $\hat \tau(x) = \frac{1}{n_x} \sum_{i: x_i= x} (\hat \mu (1, w_i, x) - \hat \mu (0, w_i, x) )$
* 하지만 이 역시도, predictor 학습을 따로 하기 때문에 아쉬움

### 1.3.2 X-Learner

* STEP1
  * Estimate $\hat \mu_1(x), \hat \mu_0(x)$
* STEP2
  * Impute ITE
    * Treatment group
      * $\hat \tau_{1, i} = Y_i(1) - \hat \mu_0 (x_i)$
    * Control group
      * $\hat \tau_{0, i} = \hat \mu_1 (x_i) - Y_i(0)$
  * Train by using imputed ITEs so that
    * Treatment group
      * $\hat \tau_{1}(x_i) : x_i \rightarrow \hat \tau_{1, i}$
    * Control group
      * $\hat \tau_{0}(x_i) : x_i \rightarrow \hat \tau_{0, i}$
* STEP3
  *  $\hat \tau(x) = g(x) \hat\tau_0(x) + (1- g(x)) \hat \tau_1(x)$
  *  where $g(x)$ is some weighting function
     * propensity score
     * etc.

# 2. Propensity Scores and IPW

* Sufficient adjustment set $W$가 고차원이면?

## 2.1 Propensity Score

* $e(W) = P(T=1 \mid W)$를 모델링하는 것이 낫다.
  * logistic regression
  * etc.

### Propensity Score Theorem

<img src="6 - Estimation.assets/image-20220822015012588.png" alt="image-20220822015012588">

* Given **positivity**,
  * $(Y(1), Y(0)) \perp T \mid W$ implies $(Y(1), Y(0)) \perp T \mid e(W)$

## 2.2 Inverse Probability Weighting (IPW)

<img src="6 - Estimation.assets/image-20220822015547077.png" alt="image-20220822015547077">

* Population을 reweight하여 $W$와 $T$의 dependency를 제거
* $\mathbb E[Y(t)] = \mathbb E[\frac{\mathbb 1 (T=t) Y}{P(t \mid W)}]$
* $\tau = \mathbb E[Y(1) - Y(0)] = \mathbb E[\frac{\mathbb 1 (T=1) Y}{e(W)}] - \mathbb E[\frac{\mathbb 1 (T=0) Y}{1-e(W)}]$
* $\hat \tau = \frac{1}{n} \sum_{i: t_i = 1} \frac{y_i}{\hat e(w_i)} - \frac{1}{n} \sum_{i: t_i = 0} \frac{y_i}{1-\hat e(w_i)}$

 ### IPW "C"ATE estimation

* See *Estimating Conditional Average Treatment Effects*

# 3. Other Methods

* Doubly robust methods
  * COM과 Propensity score를 동시에 모델링
  * 이론적으로 COM이나 IPW보다 더 빨리 수렴
  * $\frac{1}{n} \sum_i(\hat \mu(1, \hat e(w_i)) - \hat \mu (0, 1-\hat e(w_i)))$
* Double machine learning